In [17]:
from pyrtl import CompiledSimulation, reset_working_block
import torch
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from hardware_accelerators import *
from hardware_accelerators.simulation.compile import (
    ReusableCompiledSimulation,
    CompiledAccelerator,
)
from hardware_accelerators.simulation.compile import CompiledAcceleratorSimulator
from hardware_accelerators.rtllib.accelerator import CompiledAcceleratorConfig
from hardware_accelerators.nn import load_model

In [18]:
# Data transformation: convert images to tensor and normalize them
transform = transforms.Compose(
    [
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
    ]
)
# Download MNIST test data
test_dataset = datasets.MNIST(
    root="./data", train=False, download=True, transform=transform
)
test_loader = DataLoader(test_dataset, batch_size=1, shuffle=False)


def get_batch(batch_size):
    loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    batch, labels = next(iter(loader))
    return batch.reshape(batch_size, -1).numpy(), labels.numpy()


def get_activation():
    image, _ = next(iter(test_loader))
    image = image.detach().numpy().reshape(-1)
    return image

In [19]:
CHUNK_SIZE = 4

config = CompiledAcceleratorConfig(
    array_size=CHUNK_SIZE,
    activation_type=BF16,
    weight_type=BF16,
    multiplier=float_multiplier,
)

In [20]:
reset_working_block()

In [21]:
model = load_model("models/mlp_mnist.pth")
acc_sim = CompiledAcceleratorSimulator(config)

/Users/kaibreese/UCSD/dsc180b/hardware-accelerators/hardware_accelerators/nn/util.py:31: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load(model

Using precompiled library: /Users/kaibreese/UCSD/dsc180b/hardware-accelerators/hardware_accelerators/lib/wb16ab16s4/pyrtlsim.so


In [24]:
results = acc_sim.run_mlp(model, get_activation())
results

array([1.87383124e-06, 1.87383124e-06, 1.87383124e-06, 6.69541272e-06,
       1.87383124e-06, 1.87383124e-06, 1.87383124e-06, 9.99974566e-01,
       1.87383124e-06, 1.87383124e-06, 1.87383124e-06, 1.87383124e-06])

In [26]:
torch.argmax(torch.Tensor(results))

tensor(7)

In [11]:
loaded_sim.inspect_outputs()

PyrtlError: No context available. Please run a simulation step